# Stable Diffusion WebUI

阿里云 DSW 一键脚本v4 By [bilibili@秋葉aaaki](https://space.bilibili.com/12566101)

#### 更新日志

2023-6-11

- 修复了无法下载模型、无法启动的问题

2023-5-2

- 添加了插件，修改了插件安装方式
- 升级 torch 版本
- 更新了汉化
- 添加了部分插件（LoRA）
- 添加了下载文件用的代码

2023-5-3

- 疑似 torch2 会导致黑图，已取消

## 安装 第一次需要执行

In [ ]:
!apt update
!apt install -y aria2
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"

%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"

In [ ]:
## 安装常用插件，填写github链接，一行一个
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/sd-webui-infinite-image-browsing
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
"""

extensions = extension_urls.strip().split("\n")

%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

## 下载模型

In [ ]:
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

model_url = '"https://modelscope.cn/api/v1/models/dienstag/anything-v4.0/repo?Revision=master&FilePath=anything-v4.0-pruned-fp16.safetensors"'
aria2(model_url, "anything-v4.0-pruned-fp16.safetensors", "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion")

## 下载 VAE

In [ ]:
VAE_URL = '"https://modelscope.cn/api/v1/models/dienstag/anything-v4.0/repo?Revision=master&FilePath=anything-v4.0.vae.pt"'
aria2(VAE_URL, "animevae.pt", "/mnt/workspace/stable-diffusion-webui/models/VAE")

## 下载 其他文件（可选）

In [ ]:
# 这部分是可选的！！！！！有需要再执行！！！！！！！ 修改对应文本，运行即可下载文件
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

DOWNLOAD_URL = "文件链接" # 文件链接，需要下载的url直链才可以
FILE_NAME = "aki" # 下载保存的文件名
SAVE_DIR = "/mnt/workspace" # 文件保存的路径
aria2(DOWNLOAD_URL, FILE_NAME, SAVE_DIR)

## 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirror.baidu.com/pypi/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access